In [14]:
!cd

C:\Users\pongp


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import pandas as pd

#!pip install pyuca
#!pip install epitran 
#!pip install pythainlp 
#!pip install fuzzywuzzy
#!pip install python-Levenshtein #### faster for fuzzywuzzy
#!pip install openpyxl
import pyuca
import pythainlp

In [1]:
%time
df = pd.read_csv('confirmed-cases-since-271064.csv', index_col=False)
df.head(3)

Wall time: 0 ns


C:\Users\pongp\anaconda3\envs\DADS4002\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,No.,announce_date,Notified date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,Unnamed: 11
0,1859158,26/10/2021,25/10/2021,ชาย,20.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,NaN
1,1859159,26/10/2021,25/10/2021,หญิง,42.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,NaN
2,1859160,26/10/2021,25/10/2021,หญิง,33.0,ปี,Thailand,ลำปาง,ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว,ลำปาง,ห้างฉัตร,NaN


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010459 entries, 0 to 1010458
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   No.                    1010459 non-null  int64  
 1   announce_date          1010459 non-null  object 
 2   Notified date          1010459 non-null  object 
 3   sex                    951665 non-null   object 
 4   age                    923950 non-null   object 
 5   Unit                   903642 non-null   object 
 6   nationality            922053 non-null   object 
 7   province_of_isolation  1006775 non-null  object 
 8   risk                   1010380 non-null  object 
 9   province_of_onset      899173 non-null   object 
 10  district_of_onset      865756 non-null   object 
 11  Unnamed: 11            0 non-null        float64
dtypes: float64(1), int64(1), object(10)
memory usage: 92.5+ MB


In [9]:
#!pip install fastparquet
from fastparquet import write
#df.to_parquet("data1.gzip", compression='gzip')

#df.loc[df['age'] == '35/11', 'age'] = float(np.nan)

#write('outfile.parq', df)
#write('outfile_final.parq', df, compression='gzip')

#from fastparquet import ParquetFile

#pf = ParquetFile('outfile2.parq')
#df = pf.to_pandas()

# df2 = pf.to_pandas(['col1', 'col2'], categories=['col1'])

In [3]:
df = pd.read_parquet('outfile5.parq', engine = 'fastparquet')

df.head(3)

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust,risk_group
0,1859158,2021-10-26,2021-10-25,ชาย,20.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,20,Thai,อื่นๆ
1,1859159,2021-10-26,2021-10-25,หญิง,42.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,42,Thai,อื่นๆ
2,1859160,2021-10-26,2021-10-25,หญิง,33.0,ปี,Thailand,ลำปาง,ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว,ลำปาง,ห้างฉัตร,33,Thai,สถานที่และงาน


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010459 entries, 0 to 1010458
Data columns (total 14 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   no.                    1010459 non-null  object        
 1   announce_date          1010459 non-null  datetime64[ns]
 2   notified_date          1010459 non-null  datetime64[ns]
 3   sex                    951665 non-null   object        
 4   age                    923949 non-null   float64       
 5   Unit                   903642 non-null   object        
 6   nationality            921937 non-null   object        
 7   province_of_isolation  1006775 non-null  object        
 8   risk                   1010380 non-null  object        
 9   province_of_onset      926585 non-null   object        
 10  district_of_onset      864776 non-null   object        
 11  age_year               1010459 non-null  int32         
 12  nationality_adjust     92193

In [21]:
# 1.1 Drop column : "Unnamed: 11"

## check value in columns 
print(df['Unnamed: 11'].unique())   ### there is only NaN

df.drop(columns = 'Unnamed: 11', inplace=True)
df.head(1)

[nan]


,No.,announce_date,Notified date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset
0,1859158,26/10/2021,25/10/2021,ชาย,20.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง


In [22]:
# 1.2 enhancing inside data each columns

## 1.2.1 Cleansing Column : No.

df['No.'] = df['No.'].astype('str')
df.rename(columns={"No.":"no."}, inplace=True)  ### rename column

df[(df['no.'].str.isdigit() == False) | (df['no.'].str.len() != 7)] ### check ว่า มีข้อมูลใด ไม่เข้าพวกใน column นี้บ้าง

,no.,announce_date,Notified date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset


In [23]:
## 1.2.2 Cleansing Column : announce_date, Notified date

df.rename(columns={"Notified date" : "notified_date"}, inplace=True) ### rename column

column_names = df.columns

### check ว่าข้อมูลที่ใส่เข้ามาใน date เป็นช่วงที่เราใส่เข้ามาจริงๆ (ช่วงต.ค. 2021 - ก.พ.2022)
def check_date(dataframe, column_date_check, range1, range2) :
    for i in range(range1,range2) :
        for date in dataframe[column_date_check[i]].unique() :
            day, month, year = date.split('/')
            day = int(day)
            month = int(month)
            year = int(year)
            if (day<= 31) or (1 <= month <= 12) or (2021 <= year <= 2022) :
                continue
            else :
                return f"This {column_date_check[i]} is NOT correct."
        else :
            print(f"This {column_date_check[i]} is a date-input.")

check_date(df, column_names, 1, 3)

This announce_date is a date-input.
This notified_date is a date-input.


In [20]:
### change type
df['announce_date'] = pd.to_datetime(df['announce_date'], infer_datetime_format = True, errors='coerce')
df['notified_date'] = pd.to_datetime(df['notified_date'], infer_datetime_format = True, errors='coerce')

In [4]:
def isNaN(number) :     #### วิธีว่าเป็น NaN ไหม
    return number != number 

In [26]:
## 1.2.3 Cleansing Column : age, Unit

#df['age_year']  = pd.to_numeric(df['age'], downcast="integer", errors='coerce')

## ข้อมูลบางตัว ยังมี . แบบ str และ / ในข้อมูลตัวเลข จึงต้องเช็ค
def adjust_abnormal_number(dataframe, column) :
    for i in range(len(dataframe[column])) :
        if type(dataframe[column][i]) == str :
            if re.search('\D', dataframe[column][i]) :
                dataframe[column][i] = dataframe[column][i].strip()

                for ii in range(len(dataframe[column][i])) :
                        if dataframe[column][i][ii] == '.' :
                            dataframe.loc[i, column] = int(float(dataframe.loc[i, column]))
                            break
                        elif dataframe[column][i][ii] == '/' :
                            dataframe.loc[i, column] = -1
                            break
                #try :
                 #   dataframe.loc[i, column] = int(float(dataframe.loc[i, column]))
               # except :
                    #print('err')
                    
adjust_abnormal_number(df, 'age')

### check ว่ายังมีตัวเลขแปลกๆ เหลืออยู่ไหม
list_check_abnormal = []
for i in range(len(df['age'])) :
        if type(df['age'][i]) == str :
            if re.search('\D' , df['age'][i]) :
                list_check_abnormal.append(df['age'][i])
                
print(list_check_abnormal)

## new column : 'age_year'
df['age_year'] = df['age'].fillna(-1).astype(int)

### ทำการสร้าง column 'age_year' ขึ้นมาใหม่ แทนการใช้ age, Unit โดยทำการแบ่งตามดังนี้
### กรณีที่ Unit != NaN (หรือก็คือ -1)
#### - Unit : ปี ,  จะทำการแบ่งเป็น (กลุ่มที่อายุน้อยกว่า 1 ปี จะใช้ค่า = 0 ปี ในการกำหนดค่าในกลุ่มนี้) กับ (กลุ่มคนที่มีอายุมากกว่า 1 ปี จะใช้เลขตามตามนั้นไปเลย & ทำการปัดทศนิยมลง)
#### - Unit : เดือน, วัน จะอยู่ในกลุ่ม (-1)
### กรณีที่ Unit == NaN (หรือก็คือ -1)
#### - กรณี age >= 1 ; จะอยู่ในกลุ่ม (-1) 
#### - กรณี age < 1 ; จะอยู่ในกลุ่ม (กลุ่มที่อายุน้อยกว่า 1 ปี จะใช้ค่า = 0 ปี ในการกำหนดค่าในกลุ่มนี้)

for i in range(len(df['age_year'])) :
    if (df['Unit'][i] == 'ปี') & (df['age_year'][i] <= 0) & (df['age_year'][i] > -1) :
        df.loc[i, 'age_year'] = 0
    elif (df['Unit'][i] == 'ปี') & (df['age_year'][i] > 0) :
        continue
    elif ((df['Unit'][i] == 'เดือน') | (df['Unit'][i] == 'วัน')) & (df['age_year'][i] >= 0) :
        if (df['Unit'][i] == 'เดือน') :
            df.loc[i, 'age_year'] = int(df.loc[i, 'age_year'] / 12)
        if (df['Unit'][i] == 'วัน')  :
            df.loc[i, 'age_year'] = int(df.loc[i, 'age_year'] / 365)
    elif isNaN(df['Unit'][i]) & (df['age_year'][i] >= 1) :
        df.loc[i, 'age_year'] = -1
    elif isNaN(df['Unit'][i]) & (df['age_year'][i] < 1) & (df['age_year'][i] > -1) :  #### กลุ่มนี้ไม่นับ NaN ตั้งแต่แรก จึงมี (df['age_year'] > -1)
        df.loc[i, 'age_year'] = 0

[]


In [27]:
df.iloc[506120][['age', 'Unit', 'age_year']]

age          12.0
Unit        เดือน
age_year        1
Name: 506120, dtype: object

In [28]:
df[df['nationality'] == 'ไม่ระบุ']

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year
370159,2229317,2022-01-02,2022-01-01,ชาย,50.0,ปี,ไม่ระบุ,กรุงเทพมหานคร,ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว,กรุงเทพมหานคร,ดอนเมือง,50
370160,2229318,2022-01-02,2022-01-01,ชาย,10.0,ปี,ไม่ระบุ,กรุงเทพมหานคร,สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้,กรุงเทพมหานคร,ดอนเมือง,10
370161,2229319,2022-01-02,2022-01-01,ชาย,2.0,ปี,ไม่ระบุ,กรุงเทพมหานคร,สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้,กรุงเทพมหานคร,ดอนเมือง,2


In [29]:
## 1.2.3 Cleansing Column : nationality, province_of_isolation, risk, province_of_onset, district_of_onset

from fuzzywuzzy import fuzz
import pythainlp.util  #print(pythainlp.util.isthai("/"))

nationality_list = ['Afghan', 'Albanian', 'Algerian', 'American', 'Andorran', 'Angolan', 'Antiguans', 'Argentinean', 'Armenian', 'Australian', 'Austrian', 'Azerbaijani', 'Bahamian', 'Bahraini', 'Bangladeshi', 'Barbadian', 'Barbudans', 'Batswana', 'Belarusian', 'Belgian', 'Belizean', 'Beninese', 'Bhutanese', 'Bolivian', 'Bosnian_Herzegovinian', 'Brazilian', 'British', 'Bruneian', 'Bulgarian', 'Burkinabe', 'Burmese', 'Burundian', 'Chinese', 'Cambodian', 'Cameroonian', 'Canadian', 'Cape Verdean', 'Central African', 'Chadian', 'Chilean', 'Colombian', 'Comoran',  'Congolese', 'Costa Rican', 'Croatian', 'Cuban', 'Cypriot', 'Czech', 'Danish', 'Djibouti', 'Dominican', 'Dutch', 'East Timorese', 'Ecuadorean', 'Egyptian', 'Emirian', 'Equatorial Guinean', 'Eritrean', 'Estonian', 'Ethiopian', 'Fijian', 'Filipino', 'Finnish', 'French', 'Gabonese', 'Gambian', 'Georgian', 'German', 'Ghanaian', 'Greek', 'Grenadian', 'Guatemalan', 'Guinea-Bissauan', 'Guinean', 'Guyanese', 'Haitian', 'Honduran', 'Hungarian', 'I-Kiribati', 'Icelander', 'Indian', 'Indonesian', 'Iranian', 'Iraqi', 'Irish', 'Israeli', 'Italian', 'Ivorian', 'Jamaican', 'Japanese', 'Jordanian', 'Kazakhstani', 'Kenyan', 'Kittian and Nevisian', 'Kosovan','Kuwaiti', 'Kyrgyz', 'Laotian', 'Latvian', 'Lebanese', 'Liberian', 'Libyan', 'Liechtensteiner', 'Lithuanian', 'Luxembourger', 'Macedonian', 'Malagasy', 'Malawian', 'Malaysian', 'Maldivan', 'Maltese', 'Marshallese', 'Mauritanian', 'Mauritian', 'Mexican', 'Micronesian', 'Moldovan', 'Monacan', 'Mongolian', 'Moroccan', 'Mosotho', 'Motswana', 'Mozambican', 'Namibian', 'Nauruan', 'Nepalese', 'Netherlander', 'New Zealander', 'Ni-Vanuatu', 'Nicaraguan', 'Nigerian', 'Nigerien', 'North Korean', 'Northern Irish', 'Norwegian', 'Omani', 'Palestinian','Pakistani', 'Palauan', 'Panamanian', 'Papua New Guinean', 'Paraguayan', 'Peruvian', 'Polish', 'Portuguese', 'Qatari', 'Romanian', 'Russian', 'Rwandan', 'Saint Lucian', 'Salvadoran', 'Samoan', 'San Marinese', 'Sao Tomean', 'Saudi', 'Scottish', 'Senegalese', 'Serbian', 'Seychellois', 'Sierra Leonean', 'Singaporean', 'Slovakian', 'Slovenian', 'Solomon Islander', 'Somali', 'South African', 'South Korean', 'Spanish', 'Sri Lankan', 'Sudanese', 'Surinamer', 'Swazi', 'Swedish', 'Swiss', 'Syrian', 'Taiwanese', 'Tajik', 'Tanzanian', 'Thai', 'Togolese', 'Tongan', 'Trinidadian or Tobagonian', 'Tunisian', 'Turkish', 'Tuvaluan', 'Ugandan', 'Ukrainian', 'Uruguayan', 'Uzbekistani', 'Venezuelan', 'Vietnamese', 'Welsh', 'Yemenite', 'Zambian', 'Zimbabwean','Malian']
nationality_list = [i.strip().lower().capitalize() for i in nationality_list]

df['nationality'] = df['nationality'].str.strip().str.lower().str.capitalize()

df['nationality'].fillna(float(np.nan), inplace=True)

df.loc[df['nationality'] == 'ไม่ระบุ', 'nationality'] = float(np.nan)
df.loc[df['nationality'] == 'สวีเดน', 'nationality'] = 'Swedish'
df.loc[df['nationality'] == 'Au', 'nationality'] = 'Australian'
df.loc[df['nationality'] == 'UAE', 'nationality'] = 'Emirati'
df.loc[df['nationality'] == 'Uae', 'nationality'] = 'Emirati'
df.loc[df['nationality'] == 'Arabi', 'nationality'] = float(np.nan)
df.loc[df['nationality'] == 'Other', 'nationality'] = float(np.nan)
df.loc[df['nationality'] == 'United kingdom', 'nationality'] = 'British'
df.loc[df['nationality'] == 'British / english', 'nationality'] = 'British'

C:\Users\pongp\anaconda3\envs\DADS4002\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [30]:
### Adjust : putting in new column 
for i in range(len(df['nationality'])) :
    if (isNaN(df.loc[i, 'nationality']) == False) :
        for j in range(len(nationality_list)) :
            if df.loc[i, 'nationality'] == nationality_list[j] :
                df.loc[i, 'nationality_adjust'] = nationality_list[j]
    if (isNaN(df.loc[i, 'nationality']) == True) :
        df.loc[i, 'nationality_adjust'] = float(np.nan)
        
df['nationality_adjust'].fillna(float(np.nan), inplace=True)

In [127]:
### classifier 'ungroup nationality' in new column
#ungroup_nationality = df[(df['nationality_adjust'] == 'a') & (df['nationality_adjust'].isna() == False) ]
ungroup_nationality = df[(df['nationality'].isna() == False) & (df['nationality_adjust'].isna() ) ]
ungroup_nationality = ungroup_nationality['nationality'].unique()

from fuzzywuzzy import process

ungroup_list = []

for i in ungroup_nationality :
    if isNaN(i) == False :
        word_split = i.split(' / ') 
        if len(word_split) == 1  :
            word_use = process.extract(i, pd.Series(nationality_list), limit = 1) #### print(i, '&&&', word_use[0][0])  
            ungroup_list.append((i, word_use[0][0])) 
            
        elif (len(word_split) == 2) & (word_split[0] != 'British'):
            word_use = process.extract(i, pd.Series(nationality_list), limit = 1) #### print(word_split[1], '&&&', word_use)
            ungroup_list.append((i, word_use[0][0])) 

        elif (len(word_split) == 2) & (word_split[0] == 'British'):
            word_use = process.extract(word_split[1], pd.Series(nationality_list), limit = 1) #### print(word_split[1], '&&&', word_use)
            ungroup_list.append((i, word_use[0][0])) 

for i in range(len(df['nationality'])) :
    #if (isNaN(df.loc[i, 'nationality']) == False) & (df.loc[i, 'nationality_adjust'] == 'a') :
    if (isNaN(df.loc[i, 'nationality']) == False) & isNaN(df.loc[i, 'nationality_adjust']) :
        for j in ungroup_list :
            if df.loc[i, 'nationality'] == j[0] :
                df.loc[i, 'nationality_adjust'] = j[1]

In [32]:
df[df['nationality'] == 'ไม่ระบุ']

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust


In [33]:
## 1.2.4 Cleansing Column : province_of_isolation
### Checking
thai_province = [str(i) for i in df['province_of_isolation'].unique()]

thai_province.sort(key = pyuca.Collator().sort_key )
print("Old adjust",thai_province)
print(len(thai_province))

### Adjust

df['province_of_isolation'].fillna(float(np.nan), inplace=True)

for i in range(len(df['province_of_isolation'])) :
    if isNaN(i) == False :
        if df.loc[i, 'province_of_isolation'] == 'ขอนแก่น\xa0' :
            df.loc[i, 'province_of_isolation'] = 'ขอนแก่น'
        elif df.loc[i, 'province_of_isolation'] == 'กทม' :
            df.loc[i, 'province_of_isolation'] = 'กรุงเทพมหานคร'

thai_province2 = [str(i) for i in df['province_of_isolation'].unique()]
thai_province2.sort(key = pyuca.Collator().sort_key )
print("New adjust", thai_province2)
print(len(thai_province2))

Old adjust ['None', 'กทม', 'กระบี่', 'กรุงเทพมหานคร', 'กาญจนบุรี', 'กาฬสินธุ์', 'กำแพงเพชร', 'ขอนแก่น', 'ขอนแก่น\xa0', 'จันทบุรี', 'ฉะเชิงเทรา', 'ชลบุรี', 'ชัยนาท', 'ชัยภูมิ', 'ชุมพร', 'เชียงราย', 'เชียงใหม่', 'ตรัง', 'ตราด', 'ตาก', 'นครนายก', 'นครปฐม', 'นครพนม', 'นครราชสีมา', 'นครศรีธรรมราช', 'นครสวรรค์', 'นนทบุรี', 'นราธิวาส', 'น่าน', 'บึงกาฬ', 'บุรีรัมย์', 'ปทุมธานี', 'ประจวบคีรีขันธ์', 'ปราจีนบุรี', 'ปัตตานี', 'พระนครศรีอยุธยา', 'พะเยา', 'พังงา', 'พัทลุง', 'พิจิตร', 'พิษณุโลก', 'เพชรบุรี', 'เพชรบูรณ์', 'แพร่', 'ภูเก็ต', 'ภููเก็ต', 'มหาสารคาม', 'มุกดาหาร', 'แม่ฮ่องสอน', 'ยโสธร', 'ยะลา', 'ร้อยเอ็ด', 'ระนอง', 'ระยอง', 'ราชบุรี', 'ลพบุรี', 'ลำปาง', 'ลำพูน', 'เลย', 'ศรีสะเกษ', 'สกลนคร', 'สงขลา', 'สตูล', 'สมุทรปราการ', 'สมุทรสงคราม', 'สมุทรสาคร', 'สระแก้ว', 'สระบุรี', 'สิงห์บุรี', 'สุโขทัย', 'สุพรรณบุรี', 'สุราษฎร์ธานี', 'สุรินทร์', 'หนองคาย', 'หนองบัวลำภู', 'อ่างทอง', 'อำนาจเจริญ', 'อุดรธานี', 'อุตรดิตถ์', 'อุทัยธานี', 'อุบลราชธานี']
81
New adjust ['nan', 'กระบี่', 'กรุงเทพมหานคร', 'กาญ

In [34]:
## 1.2.5 Cleansing Column : province_of_onset
### Checking

from pythainlp import correct
from pythainlp.corpus import provinces  #### 77 provinces

thai_province_onset = [str(i) for i in df['province_of_onset'].unique()]

thai_province_onset.sort(key = pyuca.Collator().sort_key )
print(thai_province_onset)
print(len(thai_province_onset))

['None', 'USA', 'กระบี่', 'กระบี่ี่', 'กรุงเทพ', 'กรุงเทพมหานคร', 'กรุงเทพมหานคร.', 'กรุงเทพมหานคร. / พระนครศรีอยุธยา', 'กรุงเทพมหานคร. / ศรีสะเกษ', 'กะทู้', 'กัมพูชา', 'กาญจนบุรี', 'กาญนบุรี', 'กาฬสินธ์', 'กาฬสินธุ์', 'กาฬสินธุ์\xa0', 'กำเพงเพชร', 'กำแพง', 'กำแพงเพชร', 'กําแพงเพชร', 'กำแพงเพชร\xa0', 'ขอนแก่น', 'ขอนแก่น\xa0', 'จ.กรุงเทพมหานคร', 'จ.ราชบุรี', 'จันทบุรี', 'ฉะเชองเทราร', 'ฉะเชิงเทรา', 'ชลบุรี', 'ชลบุุรี', 'ชัยนาถ', 'ชัยนาท', 'ชัยภูมิ', 'ชัยภูมิ\xa0', 'ชุมพร', 'ชุุมพร', 'ชุุุมพร', 'เชิงเทรา', 'เชียงราย', 'เชียงใหม่', 'เชีียงใหม่', 'ไชโย', 'ดอนมดแดง', 'ดินแดง', 'ตรัง', 'ตรััง', 'ตราด', 'ตาก', 'ต่างจังหวัด', 'ทุ่งเสลี่ยม', 'นคนปฐม', 'นครนายก', 'นครปฐม', 'นครปบม', 'นครพนม', 'นครพนม\xa0', 'นครราขสีมา', 'นครราชศรีมา', 'นครราชสสีมา', 'นครราชสีมา', 'นครศรธรรมราช', 'นครศร๊ธรรมราช', 'นครศรี', 'นครศรีธรมราช', 'นครศรีธรรมราข', 'นครศรีธรรมราช', 'นครศรีฯ', 'นครศรีีธรรมราช', 'นครสวรรค์', 'นครสวรรค์\xa0', 'นครสีธรรมราช', 'นครฯ', 'นตรนายก', 'นนทบุรี', 'นนทบุรี\xa0', 'นนทุบรี', 'นนทุรี', 'น

In [35]:
### Adjust

df['province_of_onset'].fillna(float(np.nan), inplace=True)

def change_province_correct_0(dataframe, column, word_old, word_new) :
    for i in range(len(df[column])) :
        if dataframe.loc[i, column] == word_old :
            dataframe.loc[i, column] = word_new

change_province_correct_0(df, 'province_of_onset', 'ไม่ระบุ', 'กรุงเทพมหานคร')
change_province_correct_0(df, 'province_of_onset', 'ไม่มีชื่อหอพัก', 'นครศรีธรรมราช')
change_province_correct_0(df, 'province_of_onset', 'ต่างจังหวัด', float(np.nan))
change_province_correct_0(df, 'province_of_onset', 'กรุงเทพมหานคร. / พระนครศรีอยุธยา', 'ยโสธร')
change_province_correct_0(df, 'province_of_onset', 'กรุงเทพมหานคร. / ศรีสะเกษ', 'ยโสธร')
change_province_correct_0(df, 'province_of_onset', 'กะทู้', 'ภูเก็ต')
change_province_correct_0(df, 'province_of_onset', 'ประเทศกัมพูชา', 'กัมพูชา')

df['province_of_onset'] = df['province_of_onset'].str.strip()
df['province_of_onset'] = df['province_of_onset'].str.replace('.', '')

check_list_2 = []

for i in range(len(df['province_of_onset'])) :
    if isNaN(df.loc[i, 'province_of_onset']) == False :
        if df.loc[i, 'province_of_onset'] not in list(provinces()) :  #### edit เป็นชื่อจังหวัดที่ถูกต้อง
           df.loc[i, 'province_of_onset'] = correct(df.loc[i, 'province_of_onset'])  
           if df.loc[i, 'province_of_onset'] not in list(provinces()) :   #### check ว่า ยังมีตัวไหนที่ยังไม่ได้ถูกแก้ไขเป็นชื่อจังหวัดที่ถูกต้อง
               check_list_2.append(df.loc[i, 'province_of_onset'])

<ipython-input-35-8061d1455dc4>:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['province_of_onset'] = df['province_of_onset'].str.replace('.', '')


In [36]:
foreign_country = ['กัมพูชา', 'USA', 'ลาว', 'อิสราเอล', 'สวิสเซอร์แลนด์', 'ประเทศกัมพูชา','ฟินแลนด์']

def change_province_correct(dataframe, column, list_check, list_old, word_new) :
    for i in range(len(df[column])) :
        if dataframe.loc[i, column] in list_old :
            dataframe.loc[i, column] = word_new
    for j in list_old :
        if j in list_check :
            list_check.remove(j)

for i in range(len(df['province_of_onset'])) :
    if (df.loc[i, 'province_of_onset'] == 'เมือง') :
        df.loc[i, 'province_of_onset'] = df.loc[i, 'province_of_isolation']
        check_list_2.remove('เมือง')

change_province_correct(df, 'province_of_onset', check_list_2, ['กรุงเทพ', 'จกรุงเทพมหานคร'], 'กรุงเทพมหานคร')
change_province_correct(df, 'province_of_onset', check_list_2, ['กำเพงเพชร', 'กำแพง', 'กําแพงเพชร','กำเพงเพชร','กำแพง'], 'กำแพงเพชร')
change_province_correct(df, 'province_of_onset', check_list_2, ['ชลบุุรี'], 'ชลบุรี')
change_province_correct(df, 'province_of_onset', check_list_2, ['เชีียงใหม่'], 'เชียงใหม่')
change_province_correct(df, 'province_of_onset', check_list_2, ['นคนปฐม', 'นครปบม'], 'นครปฐม')
change_province_correct(df, 'province_of_onset', check_list_2, ['นคร', 'นครศรธรรมราช','นครศร๊ธรรมราช', 'นครศรีธรมราช','นครศรีธรรมราข','นครศรีฯ','นครศรีีธรรมราช','นครสีธรรมราช','นรศรีธรรมราช'], 'นครศรีธรรมราช')
change_province_correct(df, 'province_of_onset', check_list_2, ['นครราขสีมา', 'นครราชศรีมา', 'นครราชสสีมา'], 'นครราชสีมา')
change_province_correct(df, 'province_of_onset', check_list_2, ['นตรนายก'], 'นครนายก')
change_province_correct(df, 'province_of_onset', check_list_2, ['บุรี','บุรีรัมภ์','บุรีรีมย์'], 'บุรีรัมย์')
change_province_correct(df, 'province_of_onset', check_list_2, ['ปทุมฑานี','ปทุมทานี','ปทุมธาน','ปธุมธานี', 'ประทุมธานี'], 'ปทุมธานี')
change_province_correct(df, 'province_of_onset', check_list_2, ['ประจวนคีรีขันธ็','ประจวบ','ประจวนคีรีขันธ็','ประจวบคีรขันธ์','ประจวบคีรขัันธ์','ประจวบคีรีขัน','ประจวบคีรีขันธ์์','ประจวบคีรีขันธื','ประจวบคีรีขันธฺ','ประจวบคีรีขัันธ์','ประจวบคีีรีขันธ์', 'ประจวบคิรีขันธ์'], 'ประจวบคีรีขันธ์')
change_province_correct(df, 'province_of_onset', check_list_2, ['ปราจีบุรี'], 'ปราจีนบุรี')
change_province_correct(df, 'province_of_onset', check_list_2, ['พระนครศรียุธยา','อยุธยา'], 'พระนครศรีอยุธยา')
change_province_correct(df, 'province_of_onset', check_list_2, ['เมืองปัตตานี'], 'ปัตตานี')
change_province_correct(df, 'province_of_onset', check_list_2, ['ศรีษะเกษ','ศีรสะเกษ'], 'ศรีสะเกษ')
change_province_correct(df, 'province_of_onset', check_list_2, ['สมทรสงคราม'], 'สมุทรสงคราม')
change_province_correct(df, 'province_of_onset', check_list_2, ['สมุทปราการ','สมุทรปรากา','สมุทราปราการ','สุมทรปราการ','สุมุทรปราการ'], 'สมุทรปราการ')
change_province_correct(df, 'province_of_onset', check_list_2, ['สมุทรศาคร','สมุทรสาคา','สมุทราสาคร', 'สามุทรสาคร','สุมทรสาคร'], 'สมุทรสาคร')
change_province_correct(df, 'province_of_onset', check_list_2, ['สุพรรญบุรี', 'สุุพรรณบุรี'], 'สุพรรณบุรี')
change_province_correct(df, 'province_of_onset', check_list_2, ['สุรา','สุราษฎธานี','สุราษฎ์ธานี','สุราษฎร์','สุราษฏร์ธานี','สุราษร์ฎธานี','สุราษร์ธานี','สุุราษฏร์ธานี','สุุราษฯ'], 'สุราษฎร์ธานี')
change_province_correct(df, 'province_of_onset', check_list_2, ['สุรินทร์ 32150','สุรินทร์,รีสะเกษ'], 'สุรินทร์')
change_province_correct(df, 'province_of_onset', check_list_2, ['สุริะแก้ว'], 'หนองคาย')
change_province_correct(df, 'province_of_onset', check_list_2, ['หนองบัวลําภู','หนองบััวลำภู','หนอบัวลำภู'], 'หนองบัวลำภู')
change_province_correct(df, 'province_of_onset', check_list_2, ['อำนาเจริญ', 'อํานาจเจริญ','อํานาจเจริญ', 'อํานาจเจริญ'], 'อํานาจเจริญ')
change_province_correct(df, 'province_of_onset', check_list_2, ['อุดรราชธานี','อุดธานี','อุดร','อุดรธาณี', 'อุุดรธานี'], 'อุดรธานี')
#### จังหวัด โดยดูจากอำเภอ
change_province_correct(df, 'province_of_onset', check_list_2, ['สรรคบุรี', 'นัยนา'], 'ชัยนาท')
change_province_correct(df, 'province_of_onset', check_list_2, ['ดอนมดแดง'], 'อุบลราชธานี')
change_province_correct(df, 'province_of_onset', check_list_2, ['บางตา', 'ลาดกระบัง', 'หัวมัน', 'หางเสียง'], 'สมุทรปราการ')
change_province_correct(df, 'province_of_onset', check_list_2, ['สัตหีบ','บางละมุง'], 'ชลบุรี')
change_province_correct(df, 'province_of_onset', check_list_2, ['วังเวง','พรหมพิราม','ขอบเขต','ป่าไม้'], 'พิษณุโลก')
change_province_correct(df, 'province_of_onset', check_list_2, ['ช่างทอง','ไชโย','โพธิ์ทอง', 'วิเศษชัยชาญ'], 'อ่างทอง')
change_province_correct(df, 'province_of_onset', check_list_2, ['บางกะปิ', 'ดินแดง','ห้วยขวาง'], 'กรุงเทพมหานคร')
change_province_correct(df, 'province_of_onset', check_list_2, ['แม่ปะ'], 'ลำปาง')
change_province_correct(df, 'province_of_onset', check_list_2, ['ทุ่งเสลี่ยม'], 'สุโขทัย')
change_province_correct(df, 'province_of_onset', check_list_2, ['มารศรี'], 'นครศรีธรรมราช')
change_province_correct(df, 'province_of_onset', check_list_2, ['สมุย'], 'สุราษฎร์ธานี')
change_province_correct(df, 'province_of_onset', check_list_2, ['พิทยา'], 'มุกดาหาร')

In [37]:
list_3 = list()
for i in range(len(df['province_of_onset'])) :
    if (isNaN(df['province_of_onset'][i]) == False) & (df['province_of_onset'][i] not in provinces()) & (df['province_of_onset'][i] not in foreign_country):
        if df['province_of_onset'][i] not in list_3 :
            list_3.append(df['province_of_onset'][i])

print(list_3)
print(len(list_3))

['อํานาจเจริญ']
1


In [38]:
## 1.2.6 Cleansing Column : district_of_onset & province_of_onset
### if district_of_onset is not null, province_of_onset must be not null to.

df2 = pd.read_excel('ThepExcel-Thailand-Tambon.xlsx', sheet_name="DADS5001_Use", index_col=False)

pair_name = list()
pair_2_name = list()
for i in range(df2.shape[0]) :
    pair = (df2.loc[i, 'DistrictThaiShort'], df2.loc[i, 'ProvinceThai'])
    pair_2 = (df2.loc[i, 'TambonThaiShort'], df2.loc[i, 'ProvinceThai'])
    if pair not in pair_name :
        pair_name.append(pair)
    if pair_2 not in pair_2_name :
        pair_2_name.append(pair_2)

df2.head(3)

,TambonThai,TambonThaiShort,DistrictThai,DistrictThaiShort,ProvinceThai,ProvinceEng
0,แขวงพระบรมมหาราชวัง,พระบรมมหาราชวัง,เขตพระนคร,พระนคร,กรุงเทพมหานคร,Bangkok
1,แขวงวังบูรพาภิรมย์,วังบูรพาภิรมย์,เขตพระนคร,พระนคร,กรุงเทพมหานคร,Bangkok
2,แขวงวัดราชบพิธ,วัดราชบพิธ,เขตพระนคร,พระนคร,กรุงเทพมหานคร,Bangkok


In [39]:
df[(df['province_of_onset'].isna()) & (df['district_of_onset'].isna() == False)]

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust
11269,1870427,2021-10-27,2021-10-26,ชาย,39.0,ปี,Thailand,ร้อยเอ็ด,สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้,NaN,เมืองร้อยเอ็ด,39,Thai
11270,1870428,2021-10-27,2021-10-26,หญิง,65.0,ปี,Thailand,ร้อยเอ็ด,พิธีกรรมทางศาสนา,NaN,เมืองร้อยเอ็ด,65,Thai
11451,1870609,2021-10-27,2021-10-26,ชาย,19.0,ปี,Thailand,หนองบัวลำภู,อื่นๆ,NaN,นากลาง,19,Thai
11452,1870610,2021-10-27,2021-10-26,หญิง,50.0,ปี,Thailand,หนองบัวลำภู,อื่นๆ,NaN,นากลาง,50,Thai
11453,1870611,2021-10-27,2021-10-26,หญิง,49.0,ปี,Thailand,หนองบัวลำภู,อื่นๆ,NaN,เมืองหนองบัวลำภู,49,Thai
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005140,2864298,2022-02-27,2022-02-26,หญิง,56.0,ปี,Thailand,นครศรีธรรมราช,อื่นๆ,NaN,สิชล,56,Thai
1005141,2864299,2022-02-27,2022-02-26,ชาย,10.0,ปี,Thailand,นครศรีธรรมราช,อื่นๆ,NaN,สิชล,10,Thai
1005142,2864300,2022-02-27,2022-02-26,ชาย,1.0,ปี,Thailand,นครศรีธรรมราช,อื่นๆ,NaN,สิชล,1,Thai
1005143,2864301,2022-02-27,2022-02-26,หญิง,47.0,ปี,Thailand,นครศรีธรรมราช,อื่นๆ,NaN,สิชล,47,Thai


In [40]:
### Adjust

df['district_of_onset'] = df['district_of_onset'].str.strip()
df['district_of_onset'].fillna(float(np.nan), inplace=True)

list_check_4 = []
for i in range(len(df['district_of_onset']))  :
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset'] ) == False) :
        if df.loc[i, 'district_of_onset'] not in list_check_4 :
            list_check_4.append(df.loc[i, 'district_of_onset'])

change_province_correct(df, 'district_of_onset', list_check_4, ['-', '-เลือกอำเภอ-', 'ไม่ระบุ', 'มาจากต่างประเทศ', 'สเปน', 'พม่า','ต่างจังหวัด', 'ปรีดีพนมยงค์'], float(np.NaN))
change_province_correct(df, 'district_of_onset', list_check_4, ['ไทรโยคกาญจนบุรี'], 'ไทรโยค')
change_province_correct(df, 'district_of_onset', list_check_4, ['ชุนตาบง'], 'ชุมตาบง')
change_province_correct(df, 'district_of_onset', list_check_4, ['เฉลิมพระเกียติ', 'เฉลิมพระเกีรยติ'], 'เฉลิมพระเกียรติ')
change_province_correct(df, 'district_of_onset', list_check_4, ['4 ศรีบุญเรือง'], 'ศรีบุญเรือง')
change_province_correct(df, 'district_of_onset', list_check_4, ['เมืือง', 'เมืองฯ', 'เมือง ชัยนาท', 'เมืองกาญฯ','เมืองชัยนาท ชัยนาท'], 'เมือง')
change_province_correct(df, 'district_of_onset', list_check_4, ['ทุ่งใหญ่่','ทุ่งใหญ่'], 'ทุ่งใหญ่')
change_province_correct(df, 'district_of_onset', list_check_4, ['ธัญญบุรี'], 'ธัญบุรี')
change_province_correct(df, 'district_of_onset', list_check_4, ['สัฃขละบุรี','สังขละ'], 'สังขละบุรี')
change_province_correct(df, 'district_of_onset', list_check_4, ['บ้านโป่่ง'], 'บ้านโป่ง')
change_province_correct(df, 'district_of_onset', list_check_4, ['สามง่่าม'], 'สามง่าม')
change_province_correct(df, 'district_of_onset', list_check_4, ['ทุ่งสง','ทุุ่งสง'], 'ทุ่งสง')
change_province_correct(df, 'district_of_onset', list_check_4, ['เลาขวััญ'], 'เลาขวัญ')
change_province_correct(df, 'district_of_onset', list_check_4, ['เก้าเลี้ยว นครสวรรค์'], 'เก้าเลี้ยว')
change_province_correct(df, 'district_of_onset', list_check_4, ['จุุฬาภรณ์'], 'จุฬาภรณ์')
change_province_correct(df, 'district_of_onset', list_check_4, ['เดิมบางบวช'], 'เดิมบางนางบวช')
change_province_correct(df, 'district_of_onset', list_check_4, ['ตะพานนหิน'], 'ตะพานหิน')
change_province_correct(df, 'district_of_onset', list_check_4, ['ทองผาภููมิ'], 'ทองผาภูมิ')
change_province_correct(df, 'district_of_onset', list_check_4, ['ท่ามม่วง'], 'ท่าม่วง')
change_province_correct(df, 'district_of_onset', list_check_4, ['ท่ามะกาจ'], 'ท่ามะกา')
change_province_correct(df, 'district_of_onset', list_check_4, ['บ่างกะปิ'], 'บางกะปิ')
change_province_correct(df, 'district_of_onset', list_check_4, ['ปากเกล็ด'], 'ปากเกร็ด')
change_province_correct(df, 'district_of_onset', list_check_4, ['พิปููน'], 'พิปูน')
change_province_correct(df, 'district_of_onset', list_check_4, ['โพธิ์ประทับช้้าง'], 'โพธิ์ประทับช้าง')
change_province_correct(df, 'district_of_onset', list_check_4, ['สีหนุ'], 'พระสีหนุ')
change_province_correct(df, 'district_of_onset', list_check_4, ['มโนรมย์ ชัยนาท'], 'มโนรมย์')
change_province_correct(df, 'district_of_onset', list_check_4, ['วัดสิงห์ ชัยนาท'], 'วัดสิงห์') 
change_province_correct(df, 'district_of_onset', list_check_4, ['วิหารเเดง'], 'วิหารแดง') 

In [43]:
### อิงจังหวัดจากชื่ออำเภอ  >> อันนี้รันนานเกิน
### อำเภอเมือง จะยึดจาก province_of_isolation

#list_replace = ['จ.', 'อำเภอ', 'เขต', '.']

#for i in range(len(df['district_of_onset']))  :   ### run 4 hours.
#    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
#        if df.loc[i, 'district_of_onset'] == 'ทัณฑสถานวัยหนุ่ม' :
#            df.loc[i, 'province_of_onset'] = 'ปทุมธานี'

#        for j in list_replace :
#            df.loc[i, 'district_of_onset'] = df.loc[i, 'district_of_onset'].replace(j, '')
           
#            if correct(df.loc[i, 'district_of_onset']) in provinces() :
#                df.loc[i, 'province_of_onset'] = df.loc[i, 'district_of_onset']
#                break

#            if df['district_of_onset'][i] in ['เมือง','เมืองฯ', 'เมืือง'] :
#                df.loc[i, 'province_of_onset'] = df.loc[i, 'province_of_isolation']
#                break
            
#            for j in [' ', 'เมือง', 'เมืือง'] :
#                if len(df['district_of_onset'][i].split(j)) == 2 :
#                    if  correct(df.loc[i, 'district_of_onset'].split(j)[1]) in provinces() :
#                        df.loc[i, 'province_of_onset'] = df.loc[i, 'district_of_onset'].split(j)[1]
            
#            if df.loc[i,'district_of_onset'] not in ['เมือง','เมืือง', 'เมืองฯ'] :
#                for k in range(len(pair_name)) :
#                    if  df.loc[i, 'district_of_onset'] == pair_name[k][0] :
#                        df.loc[i, 'province_of_onset'] = pair_name[k][1]
#                        break

In [44]:
list_check_4.sort(key = pyuca.Collator().sort_key)
print(len(list_check_4))
list_check_4

445


['กระทุ่มแบน',
 'กรุงเทพมหานคร',
 'กรุงเทพมหานคร.',
 'กองก๋อย',
 'กะทู้',
 'กะปง',
 'กำแพงเพชร',
 'กำแพงแสน',
 'กุฉินารายณ์',
 'กุดข้าวปุ้น',
 'กุดจับ',
 'กุมภวาปี',
 'กุยบุรี',
 'กู่แก้ว',
 'เกษตรวิสัย',
 'เก้าเลี้ยว',
 'เก้าเลี้ยว จ.นครสวรรค์',
 'เกาะคา',
 'เกาะยาว',
 'แก่งกระจาน',
 'แก่งคอย',
 'โกรกพระ',
 'โกสัมพีนคร',
 'ขนอม',
 'ขาณุวรลักษบุรี',
 'ขาณุวรลักษบุรี กำแพงเพชร',
 'ขาณุวรลักษบุรี จ.กำแพงเพชร',
 'ขุนยวม',
 'เขตจตุจักร',
 'เขตดุสิต',
 'เขตตลิ่งชัน',
 'เขตบางนา',
 'เขตบึงกุ่ม',
 'เขตป้อมปราบศัตรูพ่าย',
 'เขตห้วยขวาง',
 'เขมราฐ',
 'เขาพนม',
 'เขาย้อย',
 'เขื่องใน',
 'คลองขลุง',
 'คลองท่อม',
 'คลองลาน',
 'คลองสามวา',
 'คลองหลวง',
 'ควนโดน',
 'คำตากล้า',
 'คีรีมาศ',
 'คุระบุรี',
 'โคกเจริญ',
 'โคกศรีสุพรรณ',
 'โคกสำโรง',
 'งาว',
 'จ.เชียงใหม่',
 'จตุจักร',
 'จอมทอง',
 'จันทบุรี',
 'จุฬาภรณ์',
 'เจริญศิลป์',
 'แจ้ห่ม',
 'ฉวาง',
 'เฉลิมพระเกียรติ',
 'ชนแดน จ.เพชรบูรณ์',
 'ชลบุรี',
 'ชะอวด',
 'ชะอำ',
 'ชัยบาดาล',
 'ช้างกลาง',
 'ชุมตาบง',
 'ชุมแสง',
 'เชียงคาน',
 'เชียงคำ',
 'เชีย

In [45]:
### อิงจังหวัดจากชื่ออำเภอ
### ยึดจาก รายชื่ออำเภอที่ match กับไฟล์ df2

for i in range(len(df['district_of_onset']))  :   ### run 4 hours.
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
        for k in pair_name :
            if  df.loc[i, 'district_of_onset'] == k[0] :
                df.loc[i, 'province_of_onset'] = k[1]
                break

In [73]:
### อิงจังหวัดจากชื่ออำเภอ
### อำเภอเมือง จะยึดจาก manual บางตัว

for i in range(len(df['district_of_onset']))  : 
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
        if df.loc[i, 'district_of_onset'] == 'ทัณฑสถานวัยหนุ่ม' :
            df.loc[i, 'province_of_onset'] = 'ปทุมธานี'
        if df.loc[i, 'district_of_onset'] == 'ชลบุรี' :
            df.loc[i, 'province_of_onset'] = 'ชลบุรี'
        if df.loc[i, 'district_of_onset'] in ['ป้อมปราบศัตรูพ่าย', 'ตลิ่งชัน', 'ห้วยขวาง', 'ดุสิต', 'บึงกุ่ม'] :
            df.loc[i, 'province_of_onset'] = 'กรุงเทพมหานคร'

In [47]:
### อิงจังหวัดจากชื่ออำเภอ
### อำเภอเมือง ยึดชื่อจาก province_of_isolation / ชื่อจังหวัดอยู่ใน district_of_onset

list_replace = ['จ.', 'อำเภอ', 'เขต', '.']

for i in range(len(df['district_of_onset']))  :
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
        for j in list_replace :
            df.loc[i, 'district_of_onset'] = df.loc[i, 'district_of_onset'].replace(j, '')

            if correct(df.loc[i, 'district_of_onset']) in provinces() :
                df.loc[i, 'province_of_onset'] = df.loc[i, 'district_of_onset']
                break

            if df.loc[i, 'district_of_onset'] in ['เมือง','เมืองฯ', 'เมืือง'] :
                df.loc[i, 'province_of_onset'] = df.loc[i, 'province_of_isolation']
                break

In [48]:
### อิงจังหวัดจากชื่ออำเภอ
### มีชื่ออยู่ใน district_of_onset แต่ติด in [' ', 'เมือง', 'เมืือง'] 
### จะเหลือ ['ปอยเปต', 'พระสีหนุ']ที่ไม่ใช่ในไทย และ district == 'เมือง' แต่ไม่มีชื่อจังหวัดในคอลัมภ์ไหนเลย

for i in range(len(df['district_of_onset']))  :
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
        for k in [' ', 'เมือง', 'เมืือง'] : 
            if len(df.loc[i, 'district_of_onset'].split(k)) == 2 :
                if df.loc[i, 'district_of_onset'].split(k)[1] in provinces() :
                    df.loc[i, 'province_of_onset'] = df.loc[i, 'district_of_onset'].split(k)[1]
                    break                    

In [49]:
### อิงจังหวัดจากชื่อตำบล 

list_for_tumbon = []
for i in range(len(df['district_of_onset']))  :   
    if isNaN(df.loc[i, 'province_of_onset']) & (isNaN(df.loc[i, 'district_of_onset']) == False) :
        for j in range(len(pair_2_name)) :
            if df.loc[i, 'district_of_onset'] == pair_2_name[j][0] :
                list_for_tumbon.append(pair_2_name[j][1])
            
        if len(list_for_tumbon) == 1 :
            df.loc[i, 'province_of_onset'] = list_for_tumbon[0]
        elif len(list_for_tumbon) > 1 :
            for k in list_for_tumbon :
                if k == df.loc[i, 'province_of_isolation'] :
                    df.loc[i, 'province_of_onset'] = k
        list_for_tumbon.clear()

In [74]:
#### check for ชื่ออำเภอและชื่อตำบลที่ขาดไป
df[(df['province_of_onset'].isna()) & (df['district_of_onset'].isna() == False)]['district_of_onset'].unique()

array(['ปอยเปต', 'เมือง', 'พระสีหนุ'], dtype=object)

In [52]:
df[(df['province_of_onset'].isna()==False) & (df['district_of_onset'] == 'ทัณฑสถานวัยหนุ่ม')]

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust
379285,2238443,2022-01-05,2022-01-04,None,NaN,None,NaN,นครศรีธรรมราช,อื่นๆ,ปทุมธานี,ทัณฑสถานวัยหนุ่ม,-1,NaN
391967,2251125,2022-01-07,2022-01-06,None,NaN,None,NaN,นครศรีธรรมราช,อื่นๆ,ปทุมธานี,ทัณฑสถานวัยหนุ่ม,-1,NaN
614192,2473350,2022-02-04,2022-02-03,None,NaN,None,Thailand,นครศรีธรรมราช,อื่นๆ,ปทุมธานี,ทัณฑสถานวัยหนุ่ม,-1,Thai
979330,2838488,2022-02-26,2022-02-25,ชาย,41.0,ปี,Thailand,นครศรีธรรมราช,อื่นๆ,นครศรีธรรมราช,ทัณฑสถานวัยหนุ่ม,41,Thai


In [53]:
df[df['no.'] == '2120850']

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust
261692,2120850,2021-12-02,2021-12-01,ชาย,39.0,ปี,NaN,แม่ฮ่องสอน,สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้,แม่ฮ่องสอน,กองก๋อย,39,NaN


In [72]:
df2[df2['DistrictThaiShort'] == 'บึงกุ่ม']

,TambonThai,TambonThaiShort,DistrictThai,DistrictThaiShort,ProvinceThai,ProvinceEng
106,แขวงคลองกุ่ม,คลองกุ่ม,เขตบึงกุ่ม,บึงกุ่ม,กรุงเทพมหานคร,Bangkok
107,แขวงนวมินทร์,นวมินทร์,เขตบึงกุ่ม,บึงกุ่ม,กรุงเทพมหานคร,Bangkok
108,แขวงนวลจันทร์,นวลจันทร์,เขตบึงกุ่ม,บึงกุ่ม,กรุงเทพมหานคร,Bangkok


In [55]:
df2[df2['TambonThaiShort'] == 'ป้อมปราบศัตรูพ่าย']

,TambonThai,TambonThaiShort,DistrictThai,DistrictThaiShort,ProvinceThai,ProvinceEng


In [ ]:
df[df['district_of_onset'] == 'ปรีดีพนมยงค์']

In [5]:
import pandas as pd

df = pd.read_parquet('outfile5.parq', engine = 'fastparquet')

df.head(3)

,no.,announce_date,notified_date,sex,age,Unit,nationality,province_of_isolation,risk,province_of_onset,district_of_onset,age_year,nationality_adjust,risk_group
0,1859158,2021-10-26,2021-10-25,ชาย,20.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,20,Thai,อื่นๆ
1,1859159,2021-10-26,2021-10-25,หญิง,42.0,ปี,Thailand,ลำปาง,อื่นๆ,ลำปาง,เมืองลำปาง,42,Thai,อื่นๆ
2,1859160,2021-10-26,2021-10-25,หญิง,33.0,ปี,Thailand,ลำปาง,ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว,ลำปาง,ห้างฉัตร,33,Thai,สถานที่และงาน


In [6]:
## 1.2.7 classified groups by issues of a risk column

df['risk'].fillna(float(np.nan), inplace=True)

group_1 = ['สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้','2.สัมผัสผู้ติดเชื้อ', 'สัมผัสผู้ป่วยยืนยันภายในชุมชน', 'เดินทางมาจากพื้นที่ระบาด', 'ในพื้นที่',  'เดินทางมาจากพื้นที่เสี่ยงที่มีการระบาดของโรค','เพื่อเฝ้าระวังเชื้อกลายพันธุ์', 'สัมผัสใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้ใกล้ชิดกับผู้ป่วยยืนยันรายก่อนหน้านี้', 'ติดเชื้อในพื้นที่ (สัมผัส ผป.ยืนยัน)','9.ขอตรวจหาเชื้อเอง']
group_2 = ['อยู่ระหว่างการสอบสวน','อยู่ระหว่างสอบสวนโรค']
group_3 = ['สถานที่ทำงาน', 'ทัณฑสถาน/เรือนจำ','สถานศึกษา', 'สถานบันเทิง','พิธีกรรมทางศาสนา','สนามมวย','ไปสถานที่ชุมชน','ศูนย์กักกัน ผู้ต้องกัก', 'ร่วมงานศพในหมู่บ้านทุกวัน', 'สถานประกอบการ','กลับมาจาก จ.กรุงเทพมหานคร.', 
           'ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว' ,'ไปสถานที่แออัด เช่น งานแฟร์ คอนเสิร์ต', 'นักกีฬา','อาชีพเสี่ยง เช่น ทำงานในสถานที่แออัด หรือทำงานใกล้ชิดสัมผัสชาวต่างชาติ เป็นต้น','มีอาการ มาจาก กรุงเทพมหานคร. พนักงานร้านอาหาร']
group_4 = ['การค้นหาผู้ป่วยเชิงรุกและค้นหาผู้ติดเชื้อในชุมชน', '8.สำรวจกลุ่มเสี่ยง (survey)']
group_5 = ['Phuket Sandbox','Test and Go',  'Samui plus', 'Chonburi Sandbox','Phang Nga Sandbox','BKK Sandbox', 'Krabi Sandbox', 'Phunga Sandbox', 'Karbi Sandbox',
           'Samut Prakan Sandbox','Prachuap Sandbox', 'Songkhla Sandbok','Samui Plus', 'Narathiwat Sandbox', 'Phang nga Sandbox','Rayong Sandbox',  'Phare Sandbox', 'Trat Sandbox', 'Krabi sandbox ', 'Krabi Sandbox ', 'Phrae Sandbox', 'Udon Sandbox', 'prachuap Sandbox','Trat sandbox']
group_6 = [ 'ตรวจก่อนทำหัตถการ','บุคลากรด้านการแพทย์และสาธารณสุข', 'ตรวจก่อนทำหัถตการ','บุคลากรทางการเเพทย์']
group_7 = [ 'ผู้ที่เดินทางมาจากต่างประเทศ และเข้า OQ', 'ผู้ที่เดินทางมาจากต่างประเทศ และเข้า AQ','State Quarantine','คนต่างชาติเดินทางมาจากต่างประเทศ',  'คนไทยเดินทางกลับจากต่างประเทศ','ผู้ที่เดินทางมาจากต่างประเทศ และเข้า HQ/AHQ', 'สัมผัสผู้เดินทางจากต่างประเทศ']
group_8 = ['อื่นๆ','ระบุไม่ได้', 'ปอดอักเสบ (Pneumonia)', '10.อื่นๆ']

for i in range(df.shape[0]) :
    
    if df.loc[i, 'risk'] in group_1 :
        df.loc[i, 'risk_group'] = 'สัมผัสผู้ติดเชื้อCovid'
    elif df.loc[i, 'risk'] in group_2 :
        df.loc[i, 'risk_group'] = 'อยู่ขั้นตอนการสอบสวน'
    elif df.loc[i, 'risk'] in group_3 :
        df.loc[i, 'risk_group'] = 'สถานที่และงาน'
    elif df.loc[i, 'risk'] in group_4 :
        df.loc[i, 'risk_group'] = 'การสำรวจและหาเชื้อเชิงลึก_'
    elif df.loc[i, 'risk'] in group_5 :
        df.loc[i, 'risk_group'] = 'นโยบายส่งเสริมการท่องเที่ยว'
    elif df.loc[i, 'risk'] in group_6 :
        df.loc[i, 'risk_group'] = 'กลุ่มบุคลากรทางการแพทย์'
    elif df.loc[i, 'risk'] in group_7 :
        df.loc[i, 'risk_group'] = 'กลุ่มมาจากต่างประเทศ'
    elif df.loc[i, 'risk'] in group_8 :
        df.loc[i, 'risk_group'] = 'อื่นๆ'
    elif isNaN(df.loc[i, 'risk']) :
        df.loc[i, 'risk_group'] = 'อื่นๆ'

In [8]:
df[df['risk_group'] == 'สถานที่และงาน']['risk'].unique()

array(['ไปสถานที่ชุมชน เช่น ตลาดนัด สถานที่ท่องเที่ยว', 'สถานที่ทำงาน',
       'พิธีกรรมทางศาสนา',
       'อาชีพเสี่ยง เช่น ทำงานในสถานที่แออัด หรือทำงานใกล้ชิดสัมผัสชาวต่างชาติ เป็นต้น',
       'ทัณฑสถาน/เรือนจำ', 'ไปสถานที่แออัด เช่น งานแฟร์ คอนเสิร์ต',
       'ศูนย์กักกัน ผู้ต้องกัก', 'สถานศึกษา', 'สถานบันเทิง', 'สนามมวย',
       'นักกีฬา', 'ร่วมงานศพในหมู่บ้านทุกวัน', 'สถานประกอบการ',
       'กลับมาจาก จ.กรุงเทพมหานคร.',
       'มีอาการ มาจาก กรุงเทพมหานคร. พนักงานร้านอาหาร', 'ไปสถานที่ชุมชน'],
      dtype=object)

In [57]:
หลังจากนี้ไม่เอา

NameError: name 'หลังจากนี้ไม่เอา' is not defined

In [ ]:
from fuzzywuzzy import process
# Define string and array of possible matches
string = "American"
choices = pd.Series(['americanian','america','americaa','american', 'ameri', 'aaaaaaaa'])

process.extract(string, choices, limit = 5)

In [ ]:
list_4 = []
for i in df['nationality']:
    if (isNaN(i) == False) & (i not in list_4) : #& (i not in df['nationality_adjust'].unique()):
        list_4.append(i)
list_4.sort()
print(len(list_4))
list_4

In [ ]:
threshold = 90

for i in nationality_list :
    for j in list_4 :
        if 100 > fuzz.WRatio(i, j) >= threshold :
            print(i,'&',j, fuzz.WRatio(i, j))

# if xx.split('/')[1].strip() in ['northern irish', 'scottish','welsh']
# if irish , need 100
# Malian & Somali / somalian 90  Omani & Romanian 90 South African & African 90

In [ ]:
threshold = 90

for i in range(len(df['nationality'])) :
    if isNaN(df['nationality'][i]) == False :
        for j in range(len(nationality_list)) :
            WRatio = fuzz.WRatio(i, j)

            if (WRatio == 100) & (nationality_list[j] in ['Malian', 'Omani', 'South African', 'Irish']) :
                df.loc[i, 'nationality_adjust'] = nationality_list[j]
            elif  (WRatio >= threshold) & (len(df.loc[i, 'nationality'].split(' / ')) == 2) :
                if (df.loc[i, 'nationality'].split(' / ')[0] in ['British']) :
                    df.loc[i, 'nationality_adjust'] = nationality_list[j]
            elif (WRatio >= threshold) :
                df.loc[i, 'nationality_adjust'] = nationality_list[j]
            else :
                df.loc[i, 'nationality_adjust'] = 'a'